# System of linear equations

>    To solve $Ax=b$, where $A$ is an $n\times n$ matrix.

1. Pivoting


In [ ]:
import numpy as np

## Pivoting

Consider the linear system: 
$$
\left[\begin{matrix}\epsilon & 1 \\ 1 & 1\end{matrix}\right]
\left[\begin{matrix} x \\ y\end{matrix}\right] = 
\left[\begin{matrix} 1 \\ 2\end{matrix}\right]
$$
Without pivoting, the solution is given as
$$
y_1 = \frac{2-1/\epsilon}{1-1/\epsilon}, \quad x_1 = \frac{1-y}{\epsilon}.
$$
With pivoting, the solution is given as
$$
y_2 = \frac{1-2\epsilon}{1-\epsilon}, \quad x_2 = 2-y.
$$


In [ ]:
eps = 10**(-16)
y1 = (2.0-1.0/eps)/(1.0-1.0/eps)
x1 = (1.0-y1)/eps
y2 = (1.0-2.0*eps)/(1.0-eps)
x2 = 2.0-y2
print('epsilon equals to ', eps)
print('Without pivoting: ', x1, y1)
print('With    pivoting: ', x2, y2)

epsilon equals to  1e-16
Without pivoting:  2.220446049250313 0.9999999999999998
With    pivoting:  1.0 0.9999999999999999


## Timing

To have a feeling on the time requried between the following operations:
1. matrix-vector multiplication: $A\times b$
2. Solve a linear system: $Ax=b$
3. Solve a linear system by matrix inversion: $x=A^{-1}b$

*Caution: *
矩陣與向量乘法不能直接寫 A*b, 要用要用 np.matmul(A,b)

In [ ]:
m = 2
A = m*np.identity(m) + np.random.random((m,m))
b = np.random.random((m,1))
print('A= ')
print(A)
print('b= ')
print(b)
print('A*b= ')
print(A*b)
print('np.matmul= ')
print(np.matmul(A,b))

A= 
[[2.79014313 0.02626708]
 [0.22434019 2.2497326 ]]
b= 
[[0.38308052]
 [0.00761566]]
A*b= 
[[1.06884949 0.01006241]
 [0.0017085  0.01713319]]
np.matmul= 
[[1.06904953]
 [0.10307355]]


In [ ]:
from timeit import timeit

In [ ]:
def multiplicationtime():
  return np.matmul(A, b)
def solvetime():
  return np.linalg.solve(A, b)
def solveinvtime():
  return np.linalg.inv(A)*b

Generate a $m\times m$ non-singular random matrix $A$ and a $m\times 1$ random vector $b$.

In [ ]:
m = 2000
A = m*np.identity(m) + np.random.random((m,m))
b = np.random.random((m,1))

In [ ]:
print('m=   ', m)
print('The time takes for A times b is ', timeit(stmt=multiplicationtime, number=10))
print('The time takes for solving Ax=b is ', timeit(stmt=solvetime, number=10))
print('The time takes for A^(-1) times b is ', timeit(stmt=solveinvtime, number=10))

m=    2000
The time takes for A times b is  0.032167476000267925
The time takes for solving Ax=b is  2.4206090729999232
The time takes for A^(-1) times b is  7.981646703000024
